<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0


    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-20 09:48:05
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.38 C
-------------------
Today PnL: 25.24 K (0.18%)
Current PnL: -17.45 L (-11.87%)
CY Booked + Current PnL: -8.72 L (-5.93%)
-------------------
Total profit:  3.71 L
Total loss:  -21.15 L
-------------------
Total Booked + Current PnL: 18.26 L (15.21%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.33%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 76.59 L (55.55%)
Deployed:  1.20 C
Current:  1.38 C
CAGR/XIRR %: 8.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,15.71,H-MC,14.33,165254.0,39054.0,7122.0,3.73,30.95,4.31,36.59,108.0,5.48,1.23,47.34,XY24,NTT,HEALTHCARE
68,SBIN,760.30,863.00,-11.34,M-LC,3.86,219897.0,18418.0,8796.0,-0.07,9.14,4.00,13.51,60.0,2.09,1.64,22.89,XY25,NTT,BANKS
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.89,111760.0,7448.0,9198.0,-1.16,7.14,8.23,15.96,9.0,0.81,0.83,25.24,X5K,ATH,METALS
4,APOLLOHOSP,7098.95,8285.00,-13.53,H-LC,6.95,172689.0,16512.0,9584.0,0.33,10.57,5.55,16.71,22.0,1.72,1.28,29.85,X40N,BTT,HEALTHCARE
37,HINDUNILVR,2413.81,2723.00,-6.41,H-LC,6.50,294470.0,24123.0,10513.0,0.94,8.92,3.57,12.81,24.0,2.29,2.19,23.15,XY25,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-30.43,H-MC,7.43,179300.0,-508.0,112923.0,0.08,-0.28,62.98,62.52,88.0,-0.00,1.33,33.71,XR,NTT,BANKS
58,PGHH,13388.00,18062.58,-31.08,H-MC,5.97,199635.0,-1185.0,71310.0,0.11,-0.59,35.72,34.92,80.0,-0.02,1.48,4.06,X40,ATH,FMCG
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.01,100960.0,456.0,30187.0,-0.43,0.45,29.90,30.49,79.0,0.02,0.75,33.01,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-38.34,H-LC,3.03,148712.0,-1220.0,15823.0,-0.13,-0.81,10.64,9.74,4.0,-0.08,1.11,5.38,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-21.15,H-LC,3.23,210432.0,5932.0,35479.0,-0.83,2.90,16.86,20.25,10.0,0.17,1.56,11.89,X40N,BTT,PHARMA
53,LTIM,5564.16,7230.2,-7.23,H-LC,3.72,184158.0,-16152.0,76131.0,0.09,-8.06,41.34,29.94,16.0,-0.21,1.37,27.23,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-10.77,H-LC,3.73,274786.0,9360.0,47428.0,1.11,3.53,17.26,21.40,11.0,0.20,2.04,10.40,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-10.32,H-LC,4.74,175039.0,-26347.0,113968.0,-0.21,-13.08,65.11,43.51,27.0,-0.23,1.30,21.84,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,15.71,H-MC,14.33,165254.0,39054.0,7122.0,3.73,30.95,4.31,36.59,108.0,5.48,1.23,47.34,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,14.22,H-MC,5.90,197280.0,32610.0,35353.0,2.28,19.80,17.92,41.28,89.0,0.92,1.47,29.37,X40N,ATH,AC
54,MASFIN,326.60,399.50,-15.54,H-SC,12.49,95985.0,-1995.0,23862.0,2.11,-2.04,24.86,22.32,152.0,-0.08,0.71,39.21,XR,ATH,FINANCE
42,INDIAMART,2327.09,4850.92,-49.97,H-SC,11.96,139729.0,16393.0,117372.0,1.81,13.29,84.00,108.45,119.0,0.14,1.04,38.62,AR,ATH,MISC
45,INFY,1461.46,2275.00,-19.96,H-LC,11.86,262181.0,580.0,145039.0,1.72,0.22,55.32,55.67,3.0,0.00,1.95,6.34,X40,BTT,IT


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,SONACOMS,578.05,1006.00,-37.35,M-SC,13.86,77971.0,-23188.0,98080.0,-2.13,-22.92,125.79,74.03,202.0,-0.24,0.58,9.95,AR,BTT,AUTO
70,SHALBY,261.39,327.00,911.16,M-SC,5.91,137457.0,-43686.0,89155.0,-1.39,-24.12,64.86,25.10,235.0,-0.49,1.02,10.16,XY24,NTT,HEALTHCARE
51,KPIGREEN,497.21,731.35,11.32,H-SC,11.31,132615.0,7318.0,51680.0,-1.37,5.84,38.97,47.09,141.0,0.14,0.99,65.98,MH,ATH,POWER
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.70,74726.0,-38823.0,78873.0,-1.27,-34.19,105.55,35.27,268.0,-0.49,0.56,82.89,XR,NTT,HOTELS
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.89,111760.0,7448.0,9198.0,-1.16,7.14,8.23,15.96,9.0,0.81,0.83,25.24,X5K,ATH,METALS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,100.43,M-SC,9.60,86388.0,-14389.0,14479.0,-0.63,-14.28,16.76,0.09,245.0,-0.99,0.64,13.49,OX40N,NTT,DURABLES
43,INDIGOPNTS,1407.73,1408.0,119.87,M-SC,1.94,144324.0,-30235.0,30265.0,-0.64,-17.32,20.97,0.02,221.0,-1.00,1.07,25.15,OX40N,NTT,PAINTS
84,UJJIVANSFB,52.77,60.0,83.20,H-SC,16.45,116046.0,-26433.0,45954.0,0.05,-18.55,39.60,13.70,163.0,-0.58,0.86,36.97,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-18.14,H-SC,3.00,116847.0,-26956.0,53703.0,-0.11,-18.75,45.96,18.60,149.0,-0.50,0.87,28.94,OX40N,NTT,CERAMICS
50,KANSAINER,299.63,340.0,-68.52,H-SC,3.24,217278.0,-52389.0,88715.0,-0.60,-19.43,40.83,13.47,138.0,-0.59,1.62,9.90,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,16.24,236956.0,-58978.0,152884.0,0.72,-19.93,64.52,31.73,1.0,-0.39,1.76,1.16,X40,BTT,IT
45,INFY,1461.46,2275.00,-19.96,H-LC,11.86,262181.0,580.0,145039.0,1.72,0.22,55.32,55.67,3.0,0.00,1.95,6.34,X40,BTT,IT
47,ITC,411.90,452.00,-38.34,H-LC,3.03,148712.0,-1220.0,15823.0,-0.13,-0.81,10.64,9.74,4.0,-0.08,1.11,5.38,X40,NTT,FMCG
89,VBL,492.64,671.64,-9.65,H-LC,9.64,274434.0,9394.0,86913.0,0.93,3.54,31.67,36.33,5.0,0.11,2.04,17.32,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.42,H-LC,11.64,198783.0,-17736.0,110643.0,0.57,-8.19,55.66,42.91,7.0,-0.16,1.48,4.10,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,15.71,H-MC,14.33,165254.0,39054.0,7122.0,3.73,30.95,4.31,36.59,108.0,5.48,1.23,47.34,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.68,H-LC,7.95,230614.0,46133.0,24238.0,1.14,25.01,10.51,38.14,38.0,1.90,1.71,38.26,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,15.71,H-MC,14.33,165254.0,39054.0,7122.0,3.73,30.95,4.31,36.59,108.0,5.48,1.23,47.34,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,14.22,H-MC,5.90,197280.0,32610.0,35353.0,2.28,19.80,17.92,41.28,89.0,0.92,1.47,29.37,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,11.32,H-SC,11.31,132615.0,7318.0,51680.0,-1.37,5.84,38.97,47.09,141.0,0.14,0.99,65.98,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-1.50,H-MC,7.38,206625.0,14883.0,81142.0,-0.10,7.76,39.27,50.08,99.0,0.18,1.54,15.09,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-2.52,H-MC,10.48,180495.0,29582.0,27724.0,0.76,19.60,15.36,37.97,93.0,1.07,1.34,49.87,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-2.52,H-MC,10.48,180495.0,29582.0,27724.0,0.76,19.60,15.36,37.97,93.0,1.07,1.34,49.87,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.97,H-SC,11.96,139729.0,16393.0,117372.0,1.81,13.29,84.00,108.45,119.0,0.14,1.04,38.62,AR,ATH,MISC
51,KPIGREEN,497.21,731.35,11.32,H-SC,11.31,132615.0,7318.0,51680.0,-1.37,5.84,38.97,47.09,141.0,0.14,0.99,65.98,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-42.43,M-SC,4.99,97290.0,5792.0,72958.0,0.93,6.33,74.99,86.07,223.0,0.08,0.72,40.74,XR,NTT,DURABLES
61,RAJOOENG,102.50,143.33,-30.88,H-SC,8.03,104670.0,2170.0,38665.0,1.46,2.12,36.94,39.83,114.0,0.06,0.78,3.63,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.70,74726.0,-38823.0,78873.0,-1.27,-34.19,105.55,35.27,268.0,-0.49,0.56,82.89,XR,NTT,HOTELS
6,ASIANTILES,75.41,137.00,6988.89,L-SC,6.98,77762.0,-16048.0,92661.0,-0.37,-17.11,119.16,81.67,269.0,-0.17,0.58,50.59,XR,NTT,CERAMICS
3,ALKYLAMINE,2347.98,4546.37,-3.50,H-SC,15.58,89870.0,-11093.0,105624.0,-0.67,-10.99,117.53,93.63,148.0,-0.11,0.67,33.86,SR,ATH,CHEMICALS
88,VALIANTORG,512.64,838.00,-322.08,H-SC,15.44,94367.0,-38919.0,123517.0,-0.63,-29.20,130.89,63.47,139.0,-0.32,0.70,58.18,XR,NTT,CHEMICALS
54,MASFIN,326.60,399.50,-15.54,H-SC,12.49,95985.0,-1995.0,23862.0,2.11,-2.04,24.86,22.32,152.0,-0.08,0.71,39.21,XR,ATH,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-2.52,H-MC,10.48,180495.0,29582.0,27724.0,0.76,19.60,15.36,37.97,93.0,1.07,1.34,49.87,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-7.68,H-LC,7.95,230614.0,46133.0,24238.0,1.14,25.01,10.51,38.14,38.0,1.90,1.71,38.26,X40N,NTT,FMCG
55,MEDANTA,1087.93,1486.00,15.71,H-MC,14.33,165254.0,39054.0,7122.0,3.73,30.95,4.31,36.59,108.0,5.48,1.23,47.34,XY24,NTT,HEALTHCARE
5,ASIANPAINT,2961.56,4250.00,-10.32,H-LC,4.74,175039.0,-26347.0,113968.0,-0.21,-13.08,65.11,43.51,27.0,-0.23,1.30,21.84,X40,BTT,PAINTS
15,BLUESTARCO,1646.70,2326.38,14.22,H-MC,5.90,197280.0,32610.0,35353.0,2.28,19.80,17.92,41.28,89.0,0.92,1.47,29.37,X40N,ATH,AC


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.74
1,25,41.37
2,50,71.06


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.47
LC    36.00
MC    26.52
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.66
X40      14.31
X40N     13.88
XY25     11.83
XR        9.00
AR        8.44
OX40N     7.43
X5K       2.21
MH        1.74
X200      1.37
SR        1.12
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.75
H-MC    23.61
H-SC    22.68
M-SC    13.29
M-LC     7.19
M-MC     2.58
L-SC     1.50
L-LC     1.06
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.50,0.44,30.05
IT,11.77,-17.87,76.74
FINANCE,8.14,-19.14,63.21
BANKS,7.56,-11.99,59.54
MISC,5.85,-16.96,84.01
PAINTS,5.65,-14.91,37.55
ELECTRICAL,4.72,-10.54,58.41
AUTO,4.59,-9.35,56.92
AC,3.74,3.54,29.13


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2782044.0,23
XR,1083594.0,13
AR,1073655.0,10
X40,812304.0,11
X40N,497974.0,11
XY25,458636.0,8
OX40N,457918.0,10
SR,193605.0,2
X5K,141117.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2728242.0,27
M-SC,1457498.0,17
H-MC,1201917.0,18
H-LC,1166711.0,19
M-LC,411177.0,5
M-MC,331281.0,2
L-SC,259759.0,3
L-MC,57774.0,1
L-LC,44486.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,925162.0,6
M-SC,XY24,855709.0,7
H-SC,AR,655222.0,5
H-LC,X40,524733.0,6
H-SC,XR,513632.0,6
H-MC,XY24,465870.0,5
M-MC,XY24,331281.0,2
H-LC,AR,231812.0,2
H-MC,X40,207621.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
